Request to FBI for CA crime data, agency tables, submitted through the Universal Crime Reporting (UCR) system

In [ ]:
#Define the state abbreviations list
states_abbr = ["CA"]

#Define base URL
base_url = "https://api.usa.gov/crime/fbi/cde/agency/byStateAbbr/"

#Create an empty list to store specific fields
state_abbrs = []
oris = []
agency_ids = []
agency_names = []
agency_type_names = []
county_names = []
latitudes = []
longitudes = []

#Iterate over the state abbreviations list
for state_abbr in states_abbr:
    state_url = f"{base_url}{state_abbr}?API_KEY={dataapi_key}"
    response = requests.get(state_url)
    # Check if the request was successful
    if response.status_code == 200:
        state_data = response.json()
        #Extract required data fields from above list
        for agency in state_data:
            agency_ids.append(agency.get('agency_id'))
            agency_names.append(agency.get('agency_name'))
            agency_type_names.append(agency.get('agency_type_name'))
            oris.append(agency.get('ori'))
            county_names.append(agency.get('county_name'))
            state_abbrs.append(agency.get('state_abbr'))
            latitudes.append(agency.get('latitude'))
            longitudes.append(agency.get('longitude'))
    else:
        print(f"Failed to retrieve data for state abbreviation: {state_abbr}")
        
#Create a DataFrame with the extracted fields
state_agency_data = {
    'agency_id': agency_ids,
    'agency_name': agency_names,
    'agency_type_name': agency_type_names,
    'ori': oris,
    'county_name': county_names,
    'state_abbr': state_abbrs,
    'latitude': latitudes,
    'longitude': longitudes
}
state_agency_data_df = pd.DataFrame(state_agency_data)
# Print or use the state_data list as needed
state_agency_data_df.head()

Request to FBI for CA crime data, arrest tables, submitted through the Universal Crime Reporting (UCR) system

In [ ]:
#Define the base url
base_url2 = "https://api.usa.gov/crime/fbi/cde/arrest/agency/"

#Set a lit of ori to be pulled for state_agency_data_df:ori column
ori_list = state_agency_data_df['ori']

#Create an empty dictionary to store data for each offense
offenses_data = {}

#Iterate over the list of oris
for ori in ori_list:
    
    #Query URL for searching all offenses for the current ori
    agency_all_url = f"{base_url2}{ori}/all?from=2017&to=2022&API_KEY={dataapi_key}"
    
    #GET request
    response = requests.get(agency_all_url)
    
    #Check if the request was successful
    if response.status_code == 200:
        
        # Retrieve the data for each offense from the response JSON
        data = response.json()['data']
        
        #Retrieved data to the offenses_data dictionary with ori as key
        offenses_data[ori] = data
        
#Create a DataFrame from the collected data
offenses_df = pd.DataFrame()

# Iterate over the dictionary to add data and ORI to the DataFrame
for ori, data in offenses_data.items():
    temp_df = pd.DataFrame(data)
    temp_df['ori_number'] = ori
    offenses_df = pd.concat([offenses_df, temp_df])
    
#Display the DataFrame
pprint(offenses_df)
12:40
# This is to create csv file - Export the Offense_Data into a csv
offenses_df.to_csv("output_data/offensesperori_final.csv")